In [1]:
import pandas as pd
import numpy as np
import gc
from datetime import datetime
CHUNK_SIZE = 10000

start = datetime.now()

reader_date = pd.read_csv("train_date.csv",chunksize=CHUNK_SIZE, dtype=np.float32)
reader_num = pd.read_csv("output/df_id_response.csv",chunksize=CHUNK_SIZE, dtype=np.float32)

#分割して読み込み
loop = 0
cnt_0 = 0
cnt_1 = 0
for df_chunk_date in reader_date:
    df_chunk_num = reader_num.get_chunk(CHUNK_SIZE)
    
    df_chunk = pd.merge(on="Id", left=df_chunk_date, right=df_chunk_num)
    cnt_0 = cnt_0 + len(df_chunk[df_chunk['Response'] == 0].index)
    cnt_1 = cnt_1 + len(df_chunk[df_chunk['Response'] == 1].index)

    if loop == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk])
    
    print("loop : {} / Rows = {}".format(loop, len(df.index)))

    loop = loop + 1

print("0:{}, 1:{}".format(cnt_0, cnt_1))
print("{} -> {}".format(start, datetime.now()))
print("Memory:{}".format(df.memory_usage(index=True).sum()))

loop : 0 / Rows = 10000
loop : 1 / Rows = 20000
loop : 2 / Rows = 30000
loop : 3 / Rows = 40000
loop : 4 / Rows = 50000
loop : 5 / Rows = 60000
loop : 6 / Rows = 70000
loop : 7 / Rows = 80000
loop : 8 / Rows = 90000
loop : 9 / Rows = 100000
loop : 10 / Rows = 110000
loop : 11 / Rows = 120000
loop : 12 / Rows = 130000
loop : 13 / Rows = 140000
loop : 14 / Rows = 150000
loop : 15 / Rows = 160000
loop : 16 / Rows = 170000
loop : 17 / Rows = 180000
loop : 18 / Rows = 190000
loop : 19 / Rows = 200000
loop : 20 / Rows = 210000
loop : 21 / Rows = 220000
loop : 22 / Rows = 230000
loop : 23 / Rows = 240000
loop : 24 / Rows = 250000
loop : 25 / Rows = 260000
loop : 26 / Rows = 270000
loop : 27 / Rows = 280000
loop : 28 / Rows = 290000
loop : 29 / Rows = 300000
loop : 30 / Rows = 310000
loop : 31 / Rows = 320000
loop : 32 / Rows = 330000
loop : 33 / Rows = 340000
loop : 34 / Rows = 350000
loop : 35 / Rows = 360000
loop : 36 / Rows = 370000
loop : 37 / Rows = 380000
loop : 38 / Rows = 390000
loop 

In [3]:
df_isna = df.drop('Id',axis=1).isna()
df_isna.head()

,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,L0_S0_D19,...,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263,Response
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False


In [4]:
from sklearn.model_selection import train_test_split

start = datetime.now()

## 教師データと訓練データに分離
y = df['Response']
X = df_isna

## テストセットと訓練セットに分離
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.head())
print(y_train.head())

print(X_test.head())
print(y_test.head())

print("{} -> {}".format(start, datetime.now()))

      L0_S0_D1  L0_S0_D3  L0_S0_D5  L0_S0_D7  L0_S0_D9  L0_S0_D11  L0_S0_D13  \
7472      True      True      True      True      True       True       True   
7884      True      True      True      True      True       True       True   
9721     False     False     False     False     False      False      False   
55       False     False     False     False     False      False      False   
7330      True      True      True      True      True       True       True   

      L0_S0_D15  L0_S0_D17  L0_S0_D19    ...     L3_S50_D4248  L3_S50_D4250  \
7472       True       True       True    ...             True          True   
7884       True       True       True    ...             True          True   
9721      False      False      False    ...             True          True   
55        False      False      False    ...             True          True   
7330       True       True       True    ...             True          True   

      L3_S50_D4252  L3_S50_D4254  L3_S51_D42

In [5]:
## 学習
import lightgbm as lgb

start = datetime.now()

clf = lgb.LGBMClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print(X_train.shape)
print("{} -> {}".format(start, datetime.now()))

(793110, 1157)
2018-07-14 14:40:46.256914 -> 2018-07-14 14:41:54.404458


In [6]:
from sklearn import metrics
import matplotlib.pyplot as plt

start = datetime.now()

thresholds = np.linspace(0.01,0.99,50)
y_pred = clf.predict_proba(X_test.values)[:,1]
mcc = np.array([metrics.matthews_corrcoef(y_test.values, y_pred>thr) for thr in thresholds])
plt.plot(thresholds, mcc)
print(y_test.values)
print(y_pred)
print("matthews_corrcoef = {}".format(mcc))
df_pred = pd.DataFrame({'Response':y_pred})
best_thr = thresholds[mcc.argmax()]
print("best threshold = {}".format(best_thr))

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\metrics\classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


[0. 0. 0. ... 0. 0. 0.]
[0.0055153  0.00536565 0.00536565 ... 0.0055153  0.00551289 0.0055153 ]
matthews_corrcoef = [0.10971524 0.14097843 0.14483263 0.14639781 0.14553706 0.14583629
 0.14681881 0.13998888 0.12689374 0.11914951 0.11632526 0.11257203
 0.09808276 0.08590924 0.0840087  0.07571545 0.05617551 0.03268478
 0.01843413 0.01186361 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
best threshold = 0.13


In [7]:
start = datetime.now()

#特徴量重要度を表示
for i in range(0, X_train.shape[1]):
    if 0 < clf.feature_importances_[i]:
        print("{}:{}".format(X_train.columns[i], clf.feature_importances_[i]))
print("{} -> {}".format(start, datetime.now()))

L0_S0_D1:2
L0_S3_D70:1
L0_S4_D106:2
L0_S6_D120:1
L0_S11_D280:3
L0_S19_D454:2
L0_S22_D543:2
L1_S24_D1018:2
L1_S24_D1277:4
L1_S24_D1536:1
L1_S24_D1558:11
L1_S24_D1770:4
L1_S24_D1809:1
L1_S25_D1854:5
L1_S25_D1902:3
L1_S25_D2098:1
L1_S25_D2206:4
L1_S25_D2242:8
L1_S25_D2329:2
L1_S25_D2406:1
L1_S25_D2445:3
L1_S25_D2471:4
L1_S25_D2509:17
L1_S25_D2596:1
L1_S25_D2674:1
L1_S25_D2754:1
L1_S25_D2792:1
L1_S25_D2801:2
L1_S25_D2879:1
L1_S25_D2957:3
L1_S25_D2996:1
L2_S26_D3037:20
L2_S27_D3130:18
L2_S28_D3223:7
L3_S29_D3474:7
L3_S30_D3496:3
L3_S30_D3566:23
L3_S31_D3836:20
L3_S32_D3852:39
L3_S33_D3856:9
L3_S34_D3875:22
L3_S35_D3886:5
L3_S35_D3900:3
L3_S36_D3919:7
L3_S38_D3953:19
L3_S43_D4062:2
L3_S49_D4208:1
2018-07-14 14:42:11.033225 -> 2018-07-14 14:42:11.158194


In [12]:
start = datetime.now()
reader_test_date = pd.read_csv("test_date.csv",chunksize=CHUNK_SIZE)
reader_test_num = pd.read_csv("test_numeric.csv",chunksize=CHUNK_SIZE)

#分割して読み込み
loop = 0
for df_chunk_test_date in reader_test_date:
    df_chunk_test_num = reader_test_num.get_chunk(CHUNK_SIZE)
    
    df_test_chunk = pd.merge(on="Id", left=df_chunk_test_date, right=df_chunk_test_num)
    df_test_id_chunk = df_test_chunk['Id']
    df_test_chunk = df_test_chunk.drop('Id',axis=1).isna()
    df_test_chunk = df_test_chunk.loc[:, df_isna.columns]
    
    ##予測
    ary_test_chunk_pred = clf.predict(df_test_chunk.values)
    df_test_chunk_pred = pd.DataFrame({'Id':df_test_id_chunk.values, 'Response':ary_test_chunk_pred})
    if loop == 0:
        df_test_pred = df_test_chunk_pred
    else:
        df_test_pred = pd.concat([df_test_pred, df_test_chunk_pred])
    
    print("loop : {} / Rows = {}".format(loop, len(df_test_pred.index)))
    
    loop = loop + 1



C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 0 / Rows = 10000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 1 / Rows = 20000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 116 / Rows = 1170000


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


loop : 117 / Rows = 1180000
loop : 118 / Rows = 1183748


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
df_test_pred['Response'] = df_test_pred.Response.astype('bool')
df_test_pred.head()

,Id,Response
0,1,False
1,2,False
2,3,False
3,5,False
4,8,False


In [14]:
##CSV出力
y_submit = df_test_pred
type(y_submit)
y_submit.to_csv("submit_date_is_nan.csv", columns=['Id', 'Response'], index=False)